In [33]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim
import numpy as np

In [17]:
class SiameseNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SiameseNet, self).__init__()
        self.hidden_size = hidden_size
        
        self.Linear1 = nn.Linear(input_size, hidden_size)
        self.Linear2 = nn.Linear(hidden_size, hidden_size)
        self.Output = nn.Linear(hidden_size, 1)
        
    def forward_single(self, input):
        out = F.relu(self.Linear1(input))
        out = F.relu(self.Linear2(out))
        out = self.Output(out)
        return out
    
    def forward(self, input1, input2, concat=False):
        out1 = self.forward_single(input1)
        out2 = self.forward_single(input2)
        if concat:
            return torch.cat([out1, out2], axis=1)
        return (out1, out2)
        

In [21]:
t = SiameseNet(1000, 100)

In [18]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [34]:
batch_size, input_size, hidden_size, num_classes = 64, 1000, 100, 3

x1 = torch.rand(batch_size, input_size)
x2 = torch.rand(batch_size, input_size)
x1 = Variable(x1)
x2 = Variable(x2)

label = np.random.randint(0, 1, size = (batch_size, 1))
label = Variable(torch.Tensor(label))

model = SiameseNet(input_size, hidden_size)
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

losses = []

In [35]:
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y1_pred, y2_pred = model.forward(x1, x2)

    # Compute and print loss
    loss = criterion.forward(y1_pred, y2_pred, label)
    if t % 100 == 0:
        print(t, loss.data)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 
1.00000e-03 *
  1.2658
[torch.FloatTensor of size 1]

100 
1.00000e-10 *
  9.1805
[torch.FloatTensor of size 1]

200 
1.00000e-09 *
  3.8201
[torch.FloatTensor of size 1]

300 
1.00000e-10 *
  9.3234
[torch.FloatTensor of size 1]

400 
1.00000e-12 *
  2.1943
[torch.FloatTensor of size 1]



In [ ]:
class BatchGenerator():
    def __init__(self, X, Y, batch_size):